In [6]:
import numpy as np
from collections import defaultdict

In [7]:
class Component:
    def __init__(self, name : str, node1 : int, node2 : int, current_value : float, initial_value : float):
        self.name = name
        self.node1 = node1
        self.node2 = node2
        self.current_value = current_value
        self.initial_value = initial_value
    
    def __str__(self):
        return f"{self.name} is connected between node{self.node1} and node{self.node2}. Current value is {self.current_value}. Initial value is {self.initial_value}"

In [55]:
class Simulation:
    def __init__(self, simulation_time, time_step, n, m, components):
        self.simulation_time = simulation_time
        self.time_step = time_step
        self.n = n
        self.m = m
        self.capacitors = components['C']
        self.inductors = components['I']
        self.voltage_sources = components['Vsrc']
        self.current_sources= components['Isrc']
        self.resistances = components['R']
    
    def construct_Gmat(self):
        G = np.zeros((self.n, self.n))
        for res in self.resistances:
            G[res.node1 - 1][res.node1 - 1] += (1/res.current_value)
            if res.node2 != 0:
                G[res.node2 - 1][res.node2 - 1] += (1/res.current_value)
                G[res.node1 - 1][res.node2 - 1] -= (1/res.current_value)
                G[res.node2 - 1][res.node1 - 1] -= (1/res.current_value)
        
        # Applying capacitors stamp for backward euler
        for cap in self.capacitors:
            G[cap.node1 - 1][cap.node1 - 1] += (cap.current_value/self.time_step)
            if cap.node2 != 0:
                G[cap.node2 - 1][cap.node2 - 1] += (cap.current_value/self.time_step)
                G[cap.node1 - 1][cap.node2 - 1] -= (cap.current_value/self.time_step)
                G[cap.node2 - 1][cap.node1 - 1] -= (cap.current_value/self.time_step)
        
        return G
    
    def construct_Bmat(self):
        B = np.zeros((self.n, self.m + len(self.inductors)))
        for index, voltage_source in enumerate(self.voltage_sources):
            if voltage_source.node1 != 0: 
                B[voltage_source.node1 - 1][index] = 1

            if voltage_source.node2 != 0:
                B[voltage_source.node2 - 1][index] = -1
        
        for index, ind in enumerate(self.inductors):
            if ind.node1 != 0: 
                B[ind.node1 - 1][self.m + index] = 1
            if ind.node2 != 0:
                B[ind.node2 - 1][self.m + index] = -1
        
        return B
    
    def construct_Cmat(self, B_matrix):
        return np.transpose(B_matrix)
    
    def construct_Dmat(self):
        D = np.zeros((self.m + len(self.inductors), self.m + len(self.inductors)))
        for index, ind in enumerate(self.inductors):
            D[index + self.m][index + self.m] -= (ind.current_value / self.time_step)
        
        return D
    
    def construct_Imat(self):
        i = np.zeros((self.n, 1))
        for cs in self.current_sources:
            if cs.node1 != 0:
                i[cs.node1 -1] += cs.current_value 
            if cs.node2 != 0:
                i[cs.node2 -1] += cs.current_value
        
        # Applying capacitors stamp for backward euler
        for cap in self.capacitors:
            if cap.node1 != 0:
                i[cap.node1 - 1][0] += ( (cap.current_value/self.time_step) * cap.initial_value )
            if cap.node2 != 0:
                i[cap.node2 - 1][0] -= ( (cap.current_value/self.time_step) * cap.initial_value )
        return i
    
    def construct_Emat(self):
        e = np.zeros((self.m + len(self.inductors), 1))
        for index, voltage_source in enumerate(self.voltage_sources):
            e[index - 1] = voltage_source.current_value
        
        for index, ind in enumerate(self.inductors):
            e[index + self.m - 1] -= ((ind.current_value/self.time_step) * ind.initial_value)
        
        return e
    
    def construct_Zmat(self, i_mat, e_mat):
        return np.concatenate((i_mat,e_mat), axis=0)
    
    def construct_Amat(self, G, B, C, D):
        G_C = np.concatenate((G, C), axis=0)
        B_D = np.concatenate((B, D), axis=0)
        return np.concatenate((G_C, B_D), axis=1)
    
    def solve(self):
        G = self.construct_Gmat()
        B = self.construct_Bmat()
        C = self.construct_Cmat(B_matrix=B)
        D = self.construct_Dmat()
        
        i = self.construct_Imat()
        e = self.construct_Emat()
        Z = self.construct_Zmat(i_mat=i, e_mat=e)
        
        A = self.construct_Amat(G=G, B=B, C=C, D=D)
        
        inverse_A = np.linalg.inv(A)
        X = np.matmul(inverse_A, Z)
        return X

In [56]:
def read_file(filePath):
    file = open(filePath, 'r')
    lines = file.readlines()
    
    sim_time = float(lines[0])
    num_iter = float(lines[1])
    nodes = set()
    
    circuit_components = defaultdict(list)
    
    for line in lines[2:-1]:
        name, node1, node2, current_value, initial_value = line.strip('\n').split(' ')
        nodes.add(node1)
        nodes.add(node2)
        
        component = Component(name, int(node1[-1]), int(node2[-1]), float(current_value), float(initial_value))
        circuit_components[name].append(component)
        
    return sim_time, num_iter/sim_time, circuit_components, len(nodes) - 1, len(circuit_components['Vsrc'])

In [59]:
sim_time, time_step, components, n, m = read_file('./testcases/8.txt')

In [60]:
simulation = Simulation(time_step=time_step, simulation_time=sim_time, components=components, m=m, n=n)
simulation.solve()

array([[0.00248753],
       [0.0049875 ],
       [0.99750003]])